In [1]:
import sys
import os
#instert path to modules: WORKING_DIR/py_mods/
base_dir = os.path.split(os.getcwd())[0]
mods_dir = base_dir +'/py_mods/'
sys.path.insert(0,mods_dir)
print sys.path[0]
from InitializeSAMOS import initialize_SAMOS 
from OverscanAndTrim import overscan_and_trim
from NormDivFlats import norm_div_flats
from OutlineSlits import outline_slits
from viewFITS import viewFITS
from matplotlib import pyplot as plt
#from PyrafIdentify import pyraf_identify
#from PyrafTransform import pyraf_trnsf
#the above are the main modules and functions called by the pipeline.

/Volumes/DanaExternalMacbook/PhD_Thesis/SAMOS_Project/SAMOS_pipeline/py_mods/
Main working directory is: /Volumes/DanaExternalMacbook/PhD_Thesis/SAMOS_Project/SAMOS_pipeline


/Users/DanaKoeppe/anaconda2/envs/iraf27/lib/python2.7/site-packages/astropy/table/np_utils.py:19: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from . import _np_utils
/Users/DanaKoeppe/anaconda2/envs/iraf27/lib/python2.7/site-packages/astropy/io/ascii/fastbasic.py:9: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from . import cparser


In [2]:
##INITIALIZATION##

'''
Step 1: organize the data and stores relevant information based on FITS headers.


'''
data_dir = '2017-11-30'
mask = 'LMask2'
fuel_init = initialize_SAMOS(data_dir,mask)

Scanning /Volumes/DanaExternalMacbook/PhD_Thesis/SAMOS_Project/SAMOS_pipeline/LDSS3/2017-11-30

Found field image:
['ccd8152c1.fits', 'ccd8152c2.fits']
Found flats:
['ccd8161c1.fits', 'ccd8161c2.fits', 'ccd8162c1.fits', 'ccd8162c2.fits', 'ccd8163c1.fits', 'ccd8163c2.fits']
Found lamps:
['ccd8159c1.fits', 'ccd8159c2.fits', 'ccd8160c1.fits', 'ccd8160c2.fits']
Found science:
['ccd8157c1.fits', 'ccd8157c2.fits', 'ccd8158c1.fits', 'ccd8158c2.fits']
With exposure times:
[900. 900. 900. 900.]
Keeping science:
['ccd8157c1.fits', 'ccd8157c2.fits', 'ccd8158c1.fits', 'ccd8158c2.fits']
4 science frames read
WRITTEN: this reduction step stored_LMask2/Initialize.txt 


In [3]:
##OVERSCAN AND TRIM##

'''
Step 2: collect the raw data sorted and stored in fuel_init.util._(data type)_.raw_files

- data are bias corrected and the overscan regions trimmed.
- individual chip images are stitched together, and cosmic rays cleaned.
- saved as new fits files with updated headers

Results from this step stored in fuel_init.util.intermediate_dir, and the letter `b` appended to the file name.
'''
fuel_ost = overscan_and_trim(fuel_init)

Working on /Volumes/DanaExternalMacbook/PhD_Thesis/SAMOS_Project/SAMOS_pipeline/LDSS3/2017-11-30/ccd8159c1.fits
/Volumes/DanaExternalMacbook/PhD_Thesis/SAMOS_Project/SAMOS_pipeline/LDSS3/2017-11-30/ccd8159c1.fits
Writing /Volumes/DanaExternalMacbook/PhD_Thesis/SAMOS_Project/SAMOS_pipeline/LMask2/intermediates/bLMask28159c1.fits
/Volumes/DanaExternalMacbook/PhD_Thesis/SAMOS_Project/SAMOS_pipeline/LMask2/intermediates/bLMask28159c1.fits written.
Working on /Volumes/DanaExternalMacbook/PhD_Thesis/SAMOS_Project/SAMOS_pipeline/LDSS3/2017-11-30/ccd8159c2.fits
/Volumes/DanaExternalMacbook/PhD_Thesis/SAMOS_Project/SAMOS_pipeline/LDSS3/2017-11-30/ccd8159c2.fits
Writing /Volumes/DanaExternalMacbook/PhD_Thesis/SAMOS_Project/SAMOS_pipeline/LMask2/intermediates/bLMask28159c2.fits
/Volumes/DanaExternalMacbook/PhD_Thesis/SAMOS_Project/SAMOS_pipeline/LMask2/intermediates/bLMask28159c2.fits written.
Working on /Volumes/DanaExternalMacbook/PhD_Thesis/SAMOS_Project/SAMOS_pipeline/LDSS3/2017-11-30/ccd8160

In [4]:
##Flat Field Correction##

'''
Step 3: scale each of the science and comparison arc frames to the master flat.

- normalize each individual flat by its median.
- median-combine normalized flats to create master flat.
- normalize master flat by its median.
- divide master flat out of science and arc images.

The resulting `flattened` images are corrected for pixel-to-pixel variation on the detector.

New files are stored in fuel_init.util.intermediate_dir, and the letter `f` appended to the file name.

'''

fuel_ndf = norm_div_flats(fuel_ost)

/Volumes/DanaExternalMacbook/PhD_Thesis/SAMOS_Project/SAMOS_pipeline/LMask2.db
Created list of flats in LMask2

Normalized master flat /Volumes/DanaExternalMacbook/PhD_Thesis/SAMOS_Project/SAMOS_pipeline/LMask2/intermediates/fbLMask2master_flat.fits has been created.
  Ready to be divided out of bias subtracted data frames.
Flat fielded data has been created and stored in /Volumes/DanaExternalMacbook/PhD_Thesis/SAMOS_Project/SAMOS_pipeline/LMask2/intermediates/fbLMask28158.fits
Shape of each data frame is:  (1500, 2048)
Flat fielded data has been created and stored in /Volumes/DanaExternalMacbook/PhD_Thesis/SAMOS_Project/SAMOS_pipeline/LMask2/intermediates/fbLMask28157.fits
Shape of each data frame is:  (1500, 2048)
Flat fielded data has been created and stored in /Volumes/DanaExternalMacbook/PhD_Thesis/SAMOS_Project/SAMOS_pipeline/LMask2/intermediates/fbLMask28159.fits
Shape of each data frame is:  (1500, 2048)
Flat fielded data has been created and stored in /Volumes/DanaExternalMacb

In [5]:
##Outline and Extract Slits##

'''
Step 4: use a helper_file with y-pixel slit locations to outline and cut out individual slits.

- the get_edges function uses the approximate starting slit edge pixel from the helper file to crawl 
along the image and trace the coverage of each slit.
- cutout_slit takes the traced edges, cuts them out, and straightens them.
- each slit and its corresponding calibration files are separated into directories numbered for each slit.

NOTE: This process will have to be re-written when the SAMOS instrument is installed.  Slit locations will come from
the DMD pattern and stored in the FITS headers.

'''

fuel_os = outline_slits(fuel_ndf)

Working on cutting out slits....
('/Volumes/DanaExternalMacbook/PhD_Thesis/SAMOS_Project/SAMOS_pipeline/LMask2/intermediates', 'fbLMask28158.fits')
number of cols: 2048.000000
number of rows: 1500.000000
('/Volumes/DanaExternalMacbook/PhD_Thesis/SAMOS_Project/SAMOS_pipeline/LMask2/intermediates', 'fbLMask28157.fits')
number of cols: 2048.000000
number of rows: 1500.000000
('/Volumes/DanaExternalMacbook/PhD_Thesis/SAMOS_Project/SAMOS_pipeline/LMask2/intermediates', 'fbLMask2master_flat.fits')
number of cols: 2048.000000
number of rows: 1500.000000
('/Volumes/DanaExternalMacbook/PhD_Thesis/SAMOS_Project/SAMOS_pipeline/LMask2/intermediates', 'fbLMask28159.fits')
number of cols: 2048.000000
number of rows: 1500.000000
('/Volumes/DanaExternalMacbook/PhD_Thesis/SAMOS_Project/SAMOS_pipeline/LMask2/intermediates', 'fbLMask28160.fits')
number of cols: 2048.000000
number of rows: 1500.000000
slits have been identified and excised. Ready for PyRAF identify.
WRITTEN: this reduction step stored_LMa

In [ ]:
for i in fuel_os.slits:
    print 'slit number: %s'%(getattr(i,'number'))
    print 'object name: %s'%(getattr(i,'obj'))
    print 'science frames: %s'%(getattr(i,'science'))
    print 'comparison arclamp frames: %s'%(getattr(i,'arc'))
    print 'flats: %s'%(getattr(i,'flats'))


In [ ]:
from GetFuel import get_fuel_step
wrkdir = os.path.split(os.path.abspath(os.getcwd()))[0]
fuelsavedir = os.path.join(wrkdir,'stored_LMask2')
fuel_ndf_redo = get_fuel_step(fuelsavedir,'NormFlatDiv')

In [ ]:
import OutlineSlits
reload(OutlineSlits)
from OutlineSlits import outline_slits
import SlitCutout
reload(SlitCutout)

In [ ]:
for k in vars(fuel_ost).items():
    key = k[0]
    try: 
        key_att = vars(k[1])
        keyattitems = [i for i in key_att.items()]
        print '%s: %s \n'%(key,[i[0] for i in key_att.items()])
        for kk in keyattitems:
            kkey = kk[0]
            try:
                kkey_att = vars(kk[1])
                print '%s.%s: %s \n'%(key,kkey,[j[0] for j in kkey_att.items()])
        
            
            except TypeError:
                pass
    except TypeError:
        print '%s: %s \n'%(key,k[1])


In [7]:
fuel_os.instrument.

('/Volumes/DanaExternalMacbook/PhD_Thesis/SAMOS_Project/SAMOS_pipeline/LDSS3/2017-11-30/ccd8159c1.fits',
 '/Volumes/DanaExternalMacbook/PhD_Thesis/SAMOS_Project/SAMOS_pipeline/LDSS3/2017-11-30/ccd8159c2.fits',
 '/Volumes/DanaExternalMacbook/PhD_Thesis/SAMOS_Project/SAMOS_pipeline/LDSS3/2017-11-30/ccd8160c1.fits',
 '/Volumes/DanaExternalMacbook/PhD_Thesis/SAMOS_Project/SAMOS_pipeline/LDSS3/2017-11-30/ccd8160c2.fits')